## Step

- 生成 story
- 生成每个 story 对应图片
- 生成整个故事的文字描述
- 生成视频

In [1]:
from typing import List

from pydantic import BaseModel, Field

from app.services.llm_service import chat

D:\Application\Conda\envs\party-backend\lib\site-packages\pydantic\_internal\_fields.py:151: UserWarning: Field "model_name" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
D:\Application\Conda\envs\party-backend\lib\site-packages\pydantic\_internal\_fields.py:151: UserWarning: Field "model_info" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [2]:
print(chat("hello"))

Hello! How can I assist you today?


In [28]:
epoch: int = 5


class BasePlotBrief(BaseModel):
    title: str = Field(..., title="The title of the story, brief and clear.")
    description: str = Field(..., title="The description of the plot")


class Step(BaseModel):
    plot_1: BasePlotBrief = Field(..., title="The first plot of the story")
    plot_2: BasePlotBrief = Field(..., title="The second plot of the story")
    plot_3: BasePlotBrief = Field(..., title="The third plot of the story")
    plot_4: BasePlotBrief = Field(..., title="The fourth plot of the story")


class PlotBrief(BasePlotBrief):
    url: str = Field(..., title="The URL of the image")
    final_description: str = Field(..., title="The final description of the plot")

class LLMResponse(BaseModel):
    steps: List[Step] = Field(...,
                              title=f"The step of the response, Fixed {epoch} steps.")

In [29]:
topic = "school"
system_prompt = f"""
I want to design a story for children. The themes are generally about specific aspects of life, such as 'school,' 'drawing,' and 'toy contention.'

I would like you to provide {epoch} sets of key phrases for each theme, with four options for children to choose from. These key phrases are used to string together the story. 

Some are consistent with the story logic, some are not; some are positive, some are negative. The purpose is to understand the child's emotions, personality, basic cognitive abilities about life, and whether their thinking is divergent through the children's choices.

The themes you need to generate below are as follows:
{topic}
"""
system_prompt = system_prompt.format(topic=topic, epoch=epoch)

res: LLMResponse = chat(messages=system_prompt, output_schema=LLMResponse)

In [30]:
for item in res.steps:
    print(item)
    print("----------")

plot_1=BasePlotBrief(title='The first day of school', description='You see your best friend waiting for you outside the school gate.') plot_2=BasePlotBrief(title='First class begins', description='Your teacher asked you to introduce yourself in front of the class.') plot_3=BasePlotBrief(title='Lunchtime', description='You find out that somebody ate your favorite snack from your lunchbox.') plot_4=BasePlotBrief(title='Recess', description='You join in a game with new friends and have so much fun.')
----------
plot_1=BasePlotBrief(title="The school's pet", description='Your school has a cute rabbit as a class pet, and you get to take care of it for a day.') plot_2=BasePlotBrief(title='Math class', description='You struggle with a difficult math problem but finally figure it out with the help of a classmate.') plot_3=BasePlotBrief(title='Art class', description='You draw a picture of your family, and the teacher praises you for your creativity.') plot_4=BasePlotBrief(title='Library time',

In [32]:
print(len(res.steps))

4


## Generate All Images

In [37]:
from app.services.img_gen_service import fal

plot_map: List[List[PlotBrief]] = []

def generate_img():
    plot_map = []

    for i in range(epoch):
        tmp: List[PlotBrief] = []

        img_1 = fal.generate(prompt=res.steps[i].plot_1.description)
        tmp.append(PlotBrief(url=img_1, title=res.steps[i].plot_1.title,
                             description=res.steps[i].plot_1.description))

        img_2 = fal.generate(prompt=res.steps[i].plot_1.description)
        tmp.append(PlotBrief(url=img_2, title=res.steps[i].plot_2.title,
                             description=res.steps[i].plot_2.description))

        img_3 = fal.generate(prompt=res.steps[i].plot_1.description)
        tmp.append(PlotBrief(url=img_3, title=res.steps[i].plot_3.title,
                             description=res.steps[i].plot_3.description))

        img_4 = fal.generate(prompt=res.steps[i].plot_1.description)
        tmp.append(PlotBrief(url=img_4, title=res.steps[i].plot_4.title,
                             description=res.steps[i].plot_4.description))

        plot_map.append(tmp)

In [ ]:
generate_img()

In [39]:
from PIL import Image
import requests
from io import BytesIO


def show_all():
    # 1 as a demo, default is epoch
    for i in range(1):
        for j in range(4):
            response = requests.get(plot_map[i][j].url)
            img = Image.open(BytesIO(response.content))
            img.show()
            print(plot_map[i][j].title)
            print(plot_map[i][j].description)
            print("----------")

In [40]:
# show_all()

The first day of school
You see your best friend waiting for you outside the school gate.
----------
First class begins
Your teacher asked you to introduce yourself in front of the class.
----------
Lunchtime
You find out that somebody ate your favorite snack from your lunchbox.
----------
Recess
You join in a game with new friends and have so much fun.
----------


## Generate the story
需要选择路径，四个图片代表0 1 2 3，则一个故事可能选择为 [1, 0, 2, 3, 1]


In [47]:
choice = [1, 1, 2, 3]

appended_description = []

system_prompt = """
现在你会得到一个长度为n的数组，里面的每一个元素代表了当前故事的一段描述，这个数组的每个元素append起来构成了一个时序的故事描述。

现在你需要根据给出的故事描述，生成一段完整的故事，并且返回一个长度为n的数组，里面的每一个元素代表了当前故事的一段描述。
"""

def generate_story():
    for i in range(epoch):
        # append 每一个 description
        item_description: str = plot_map[i][choice[i]].title + plot_map[i][choice[i]].description
    
    

    chat("")
        

In [48]:
generate_story()

The first day of school
You see your best friend waiting for you outside the school gate.
First class begins
Your teacher asked you to introduce yourself in front of the class.
----------
The school's pet
Your school has a cute rabbit as a class pet, and you get to take care of it for a day.


IndexError: list index out of range